In [ ]:
from functions import *

import tensorflow as tf
from keras import layers, models
from keras.utils import to_categorical
import os
import numpy as np
import cv2
import random
from exif import Image


In [ ]:
#Comprobamos si nuestras imagenes tienen metadatos para obtener de una amnera facil las etiquetas.

img_path = '../data/train/Letter_A_may_2.png'

with open(img_path, 'rb') as img_file:
    image = Image(img_file)

print(image.has_exif)

Como vemos las imagenes de nuestro dataset no tienen metadatos, por lo tanto tendremos que buscar las etiquetas para nuestras neuronas de salida de otra manera.

In [ ]:
#Queremos que nuestras imagenes tengan unas dimensiones no muy grandes.
width = 32
height = 32


train_path = '../data/train'
test_path = '../data/test'

train_x = x_generator(train_path, width, height)
train_labels = labels_generator(train_path)
train_y = y_generator(train_path, train_labels)

In [ ]:
#Pasamos nuestros datos de train a arrays.

x_data_train = np.array(train_x)
y_data_train = np.array(train_y, dtype=np.dtype(np.float32))

print(x_data_train.shape[0])
print(y_data_train.shape)





In [ ]:
print('La cantidad de neuronas de salida es de:',len(train_labels))
print(train_labels)

In [ ]:
x_data_train = np.array(train_x).reshape(x_data_train.shape[0], width, height, 3)

num_classes = len(train_labels)

#y_data_train = to_categorical(y_data_train) Ya la tenemos categorizada.
print(y_data_train.shape)

Ya hemos obtenido nuestras etiquetas.

In [ ]:


from tensorflow.keras.preprocessing.image import ImageDataGenerator

rango_rotacion = 30
mov_ancho = 0.25
mov_alto = 0.25
rango_acercamiento=[0.5,1.5]

datagen = ImageDataGenerator(
    rotation_range = rango_rotacion,
    width_shift_range = mov_ancho,
    height_shift_range = mov_alto,
    zoom_range=rango_acercamiento,

)

datagen.fit(x_data_train)


In [ ]:
#Los datos para entrenar saldran del datagen, de manera que sean generados con las transformaciones que indicamos
data_gen_entrenamiento = datagen.flow(x_data_train, y_data_train, batch_size=32)

In [ ]:
#Normalizamos los datos.

x_data_train_normalized = x_data_train.astype('float32') / 255


In [ ]:
model = tf.keras.Sequential([
    layers.Input(shape=(width,height,3)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(100, activation='relu'),
    layers.Dense(num_classes, activation="softmax")
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_data_train_normalized, y_data_train, epochs= 80, batch_size = 32, steps_per_epoch=int(np.ceil(60000 / float(32))))

model.save('model_80_sigmoid_softmax_relu_32.keras')

In [ ]:
#Predicciones
model_test = models.load_model('model_60_sigmoid.keras')

img = cv2.imread('../data/test/Letter_b_min_186.png')
img_resized = cv2.resize(img,(28,28))
result = model_test.predict(np.array([img_resized]))
print(result)